### Submitted by Group 11:
##### Yaswanth - 2003287
##### Mohamad - 2004060
##### Sreelakshmi - 2004055
##### Abhay - 2004349

# Data Description:
We are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

## Files
train.csv - historical data including Sales,
test.csv - historical data excluding Sales,
sample_submission.csv - a sample submission file in the correct format,
store.csv - supplemental information about the stores.

## Problem Statement:
Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. We need to predict sales based on their unique circumstances.

## Data fields:

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing
import math
import seaborn as sns
from pandasql import sqldf
import matplotlib.pyplot as plt
#for removing warnings
import warnings
warnings.filterwarnings('ignore')
from keras import backend as K
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

KeyboardInterrupt: 

In [ ]:
# Reading files
train=pd.read_csv("train.csv")
print(train.shape)
train.head()

In [ ]:
test=pd.read_csv("test.csv")
print(test.shape)
test.head()

In [ ]:
store_df=pd.read_csv("store.csv")
print(store_df.shape)
store_df.head()

Checking for null values

In [ ]:
train.info() # no null values in train data
print("----------------------------------------------")
test.info() #few null values in open

#### Train Data Analysis

In [ ]:
train.describe()

In [ ]:
train['Date'] = pd.to_datetime(train['Date'])

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,4))
sns.countplot(x='Open',hue='DayOfWeek', data=train,palette="husl", ax=ax1)

Many stores are closed on Sundays 

In [ ]:
# Date

# Create Year and Month columns
train['Year']  = train['Date'].apply(lambda x: int(str(x)[:4]))
train['Month'] = train['Date'].apply(lambda x: int(str(x)[5:7]))

test['Year']  = test['Date'].apply(lambda x: int(str(x)[:4]))
test['Month'] = test['Date'].apply(lambda x: int(str(x)[5:7]))

# Assign Date column to Date(Year-Month) instead of (Year-Month-Day)
train['Date'] = train['Date'].apply(lambda x: (str(x)[:7]))
test['Date']  = test['Date'].apply(lambda x: (str(x)[:7]))

# group by date and get average sales, and percent change
avg_sales   = train.groupby('Date')["Sales"].mean()
pct_change_sales = train.groupby('Date')["Sales"].sum().pct_change()

fig, (axis1,axis2) = plt.subplots(2,1,sharex=True,figsize=(15,8))

# plot average sales over time(year-month)
ax1 = avg_sales.plot(legend=True,ax=axis1,marker='o',title="Average Sales")
ax1.set_xticks(range(len(avg_sales)))
ax1.set_xticklabels(avg_sales.index.tolist(), rotation=90)

# plot precent change for sales over time(year-month)
ax2 = pct_change_sales.plot(legend=True,ax=axis2,marker='o',rot=90,colormap="summer",title="Sales Percent Change")

We can see that sales during december are at peak

In [ ]:
# Plot average sales and customers over years
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,4))

sns.barplot(x='Year', y='Sales', data=train, ax=axis1)
sns.barplot(x='Year', y='Customers', data=train, ax=axis2)

In [ ]:
# Plot average sales and customers over days of week
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,4))

sns.barplot(x='DayOfWeek', y='Sales', data=train, ax=axis1)
sns.barplot(x='DayOfWeek', y='Customers', data=train, ax=axis2) 

Sales and Customers on Sunday are lowest as many stores are closed 

In [ ]:
# Plot average sales and customers over months
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,4))

sns.barplot(x='Month', y='Sales', data=train, ax=axis1)
sns.barplot(x='Month', y='Customers', data=train, ax=axis2)

Sales and Customers are comparatively higher in December as compared to other months

In [ ]:
# Plot average sales and customers with/without promo
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,4))

sns.barplot(x='Promo', y='Sales', data=train, ax=axis1)
sns.barplot(x='Promo', y='Customers', data=train, ax=axis2)

We can clearly see without promo the store doesn't stand a chance against stores with promo

In [ ]:
# StateHoliday has values 0 & "0", So, we need to merge values with 0 to "0"
train["StateHoliday"]= train["StateHoliday"].replace(0, "0")

sns.countplot(x='StateHoliday',data=train)

# Plot average sales on StateHoliday
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,4))

sns.barplot(x='StateHoliday', y='Sales', data=train, ax=axis1)
filt = (train["StateHoliday"] != "0") & (train["Sales"] > 0) # we are taking sales which are more than 0
sns.barplot(x='StateHoliday', y='Sales', data=train[filt], ax=axis2)
plt.show()

In [ ]:
# Combining a,b and c type stores so as to reduce the bias
train["StateHoliday"] = train["StateHoliday"].map({0: 0, "0": 0, "a": 1, "b": 1, "c": 1})
test["StateHoliday"] = test["StateHoliday"].map({0: 0, "0": 0, "a": 1, "b": 1, "c": 1})

fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,4))
plt.title('All Sales of stores')
sns.barplot(x='StateHoliday', y='Sales', data=train, ax=axis1)
filt = (train["Sales"] > 0) # we are taking sales which are more than 0
sns.barplot(x='StateHoliday', y='Sales', data=train[filt], ax=axis2)
plt.title('Average Sales of stores which are open')

From the above graphs we can clearly see that the stores which are 'OPEN' during stateholidays have high sales

In [ ]:
# Visualizing Sales over SchoolHoliday
sns.countplot(x='SchoolHoliday',data=train)

# Plot average sales on StateHoliday
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,4))

sns.barplot(x='SchoolHoliday', y='Sales', data=train, ax=axis1)

sns.barplot(x='SchoolHoliday', y='Customers', data=train, ax=axis2)

We can clearly see that the sales & customers during School Holiday are more compared to normal days

In [ ]:
train["Sales"].plot(kind='hist',bins=70,xlim=(0,15000))

There are mostly 0's in this plot because the stores were closed

STORE DATA

In [ ]:
store_df.head()

In [ ]:
store_df.info() # many null values
print("----------------------------------------------")

In [ ]:
store_df['PromoInterval'].value_counts()

### Data Preprocessing

In [ ]:
# Merging train and store_df
train_store = train.merge(store_df,left_on=['Store'], right_on=['Store'],how='left')
print(train_store.shape)
train_store.head()

In [ ]:
test_store = test.merge(store_df,left_on=['Store'], right_on=['Store'],how='left')
print(test_store.shape)
test_store.head()

In [ ]:
train_store.info()

In [ ]:
test_store.info()

In [ ]:
# Checking correlation bw different variables
plt.figure(figsize=(25,25))
sns.heatmap(train_store.corr(),vmax=.7,cbar=True,annot=True)

The highly correlated features with the target variable(Sales) are :- Promo, Open, Customers, DayOfWeek

**Treating Missing Values**

In [ ]:
train_store.isnull().sum()

In [ ]:
# Plot average sales on StateHoliday
fig, (axis1,axis2,axis3,axis4,axis5,axis6) = plt.subplots(1,6,figsize=(20,4))
plt.title('CompetitionDistance')
sns.boxplot( train_store['CompetitionDistance'],ax=axis2)
plt.title('CompetitionOpenSinceMonth')
sns.boxplot( train_store['CompetitionOpenSinceMonth'], ax=axis3)
plt.title('CompetitionOpenSinceYear')
sns.boxplot( train_store['CompetitionOpenSinceYear'], ax=axis4)
plt.title('Promo2SinceWeek')
sns.boxplot( train_store['Promo2SinceWeek'], ax=axis5)
plt.title('Promo2SinceYear')
sns.boxplot( train_store['Promo2SinceYear'], ax=axis6)

You can see there are lot of outliers

In [ ]:
# Week of year and calculating promo
train_store['WeekOfYear'] = pd.DatetimeIndex(train_store['Date']).weekofyear
train_store['PromoOpen'] = 12 * (train_store.Year - train_store.Promo2SinceYear) + \
        (train_store.WeekOfYear - train_store.Promo2SinceWeek) / 4.0
train_store['PromoOpen'] = train_store.PromoOpen.apply(lambda x: x if x > 0 else 0)
train_store.loc[train_store.Promo2SinceYear == 0, 'PromoOpen'] = 0

In [ ]:
test_store['WeekOfYear'] = pd.DatetimeIndex(test_store['Date']).weekofyear
test_store['PromoOpen'] = 12 * (test_store.Year - test_store.Promo2SinceYear) + \
        (train_store.WeekOfYear - test_store.Promo2SinceWeek) / 4.0
test_store['PromoOpen'] = test_store.PromoOpen.apply(lambda x: x if x > 0 else 0)
test_store.loc[test_store.Promo2SinceYear == 0, 'PromoOpen'] = 0

In [ ]:
train_store['CompetitionOpen'] = 12 * (train_store.Year - train_store.CompetitionOpenSinceYear) + (train_store.Month - train_store.CompetitionOpenSinceMonth)
test_store['CompetitionOpen'] = 12 * (test_store.Year - train_store.CompetitionOpenSinceYear) + (test_store.Month - test_store.CompetitionOpenSinceMonth)

In [ ]:
#Replacing null values with median
med_comp_month = train_store['PromoOpen'].astype('float').median(axis=0)
train_store['PromoOpen'].replace(np.nan,math.floor(med_comp_month),inplace=True)

med_comp_month = train_store['PromoOpen'].astype('float').median(axis=0)
test_store['PromoOpen'].replace(np.nan,math.floor(med_comp_month),inplace=True)

med_comp_month = train_store['CompetitionOpen'].astype('float').median(axis=0)
train_store['CompetitionOpen'].replace(np.nan,math.floor(med_comp_month),inplace=True)

med_comp_month = train_store['CompetitionOpen'].astype('float').median(axis=0)
test_store['CompetitionOpen'].replace(np.nan,math.floor(med_comp_month),inplace=True)

I am replacing with Median (instead of mean due to outliers) as they are not highly correlated to the dependent variable

In [ ]:
train_store.isnull().sum() # no null values

#### Train and Store data Analysis

In [ ]:
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,5))

#plt.title('StoreType vs Sales')
sns.barplot(x='StoreType', y='Sales', data=train_store, order=['a','b','c', 'd'],ax=axis2)
sns.countplot(x='StoreType',data=train_store, order=['a','b','c', 'd'],ax=axis1)
#plt.title('StoreType vs Customers')
#sns.barplot(x='StoreType', y='Customers', data=train_store, order=['a','b','c', 'd'], ax=axis2)

You can see that the 'b-type' stores are less but have high sales and volume, while 'a-type' stores are high in number but have relatively low sales and volume 

In [ ]:
#plt.title('Assortment')
#sns.countplot(x='Assortment', data=train_store, order=['a','b','c'], ax=axis1)
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,5))
#plt.title('Assortment vs Sales')
sns.countplot(x='Assortment', data=train_store, order=['a','b','c'], ax=axis1)
sns.barplot(x='Assortment', y='Sales', data=train_store, order=['a','b','c'], ax=axis2)
#plt.title('Assortment vs Customers')
#sns.barplot(x='Assortment', y='Customers', data=train_store, order=['a','b','c'], ax=axis2)

In [ ]:
plt.title('Promo2')
sns.countplot(x='Promo2', data=train_store)

fig, (axis1,axis2) = plt.subplots(1,2,figsize=(10,4))

plt.title('Promo2 vs Sales')
sns.barplot(x='Promo2', y='Sales', data=train_store, ax=axis1)
plt.title('Promo2 vs Customers')
sns.barplot(x='Promo2', y='Customers', data=train_store, ax=axis2)

In [ ]:
sns.distplot(train_store['Customers'],color='Black')
train_store['Customers'].skew()

In [ ]:
sns.distplot(train_store['Sales'],color='Black')
train_store['Sales'].skew()

In [ ]:
# Plotting correlations
num_feat=train_store.columns[train_store.dtypes!=object]
num_feat=num_feat[1:-1] 
labels = []
values = []
for col in num_feat:
    labels.append(col)
    values.append(np.corrcoef(train_store[col].values, train_store['Sales'].values)[0,1])
ind = np.arange(len(labels))
width = 0.9
fig, ax = plt.subplots(figsize=(10,5))
rects = ax.barh(ind, np.array(values), color='red')
ax.set_yticks(ind+((width)/2.))
ax.set_yticklabels(labels, rotation='horizontal')
ax.set_xlabel("Correlation coefficient")
ax.set_title("Correlation Coefficients w.r.t Sales")

### Feature Engineering & Selection of Train and Test Data

In [ ]:
train_store.head()

In [ ]:
# Creating new columns Average Customers and Sales Per Customerfrom pandasql import sqldf
avg_customer = sqldf(
      """
      SELECT
      Store,
      DayOfWeek,
      sum(case when Customers is not null then Sales/Customers else 0 end) as SpC,
      round(avg(Customers)) Avg_Customers
      from train_store
      group by Store,DayOfWeek
      """
    )
    
test_store = sqldf(
      """
      SELECT
      t.*,
      ac.SpC,
      ac.Avg_Customers
      from test_store t
      left join avg_customer ac on t.Store = ac.Store and t.DayOfWeek = ac.DayOfWeek
      """
    )
train_store = sqldf(
      """
      SELECT
      t.*,
      ac.SpC,
      ac.Avg_Customers
      from train_store t
      left join avg_customer ac on t.Store = ac.Store and t.DayOfWeek = ac.DayOfWeek
      """
    )

In [ ]:
test_store.head()

In [ ]:
train_store.isnull().sum()

In [ ]:
# Create dummy varibales for DayOfWeek
train_dummies  = pd.get_dummies(train_store['DayOfWeek'], prefix='Day')
train_dummies.drop(['Day_7'], axis=1, inplace=True)

test_dummies = pd.get_dummies(test_store['DayOfWeek'],prefix='Day')
test_dummies.drop(['Day_7'], axis=1, inplace=True)

train_store = train_store.join(train_dummies)
test_store = test_store.join(test_dummies)

In [ ]:
# Create dummy varibales for Assortment
train_store_dummies  = pd.get_dummies(train_store['Assortment'], prefix='Assortment')
train_store_dummies.drop(['Assortment_c'], axis=1, inplace=True)

test_store_dummies = pd.get_dummies(test_store['Assortment'],prefix='Assortment')
test_store_dummies.drop(['Assortment_c'], axis=1, inplace=True)

train_store = train_store.join(train_store_dummies)
test_store = test_store.join(test_store_dummies)

In [ ]:
# Create dummy varibales for Storetype
train_store_dummies  = pd.get_dummies(train_store['StoreType'], prefix='StoreType')
train_store_dummies.drop(['StoreType_d'], axis=1, inplace=True)

test_store_dummies = pd.get_dummies(test_store['StoreType'],prefix='StoreType')
test_store_dummies.drop(['StoreType_d'], axis=1, inplace=True)

train_store = train_store.join(train_store_dummies)
test_store = test_store.join(test_store_dummies)

In [ ]:
#Dropping unnecessary columns from train and test set
train_store.drop(['Customers','CompetitionOpenSinceYear','CompetitionOpenSinceMonth','CompetitionDistance','Promo2SinceWeek','Promo2SinceYear','PromoInterval','WeekOfYear','Year','StoreType','Assortment','Date'],axis=1,inplace=True)
train_store['Open'] = train_store['Open'].astype(float)
test_store.drop(['Year','CompetitionOpenSinceYear','CompetitionOpenSinceMonth','CompetitionDistance','Promo2SinceWeek','Promo2SinceYear','PromoInterval','Promo2SinceWeek','Promo2SinceYear','PromoInterval','WeekOfYear','WeekOfYear','StoreType','Assortment','Date'],axis=1,inplace=True)

In [ ]:
print(train_store.info())
print(test_store.info())

In [ ]:
test_store[test_store['Open'].isnull()]

As you can see all the null values of open are when week is not 7 and StateHoliday and SchoolHoliday are 0's. So replacing them with 1's

In [ ]:
# fill NaN values in test with Open=1
test_store["Open"].fillna(1, inplace = True) 

In [ ]:
# Dropping DayOfWeek
train_store.drop(['DayOfWeek'], axis=1,inplace=True)
test_store.drop(['DayOfWeek'], axis=1,inplace=True)

removing all rows(stores) that were closed as the sales are 0 when store is closed and it tend to make the model unstable

In [ ]:
# remove all rows(store,date) that were closed
train_store= train_store[train_store["Open"] != 0]

Saving id's of those stores which were closed so we can put 0 in their respective sales column

In [ ]:
# Saving id's of those stores which were closed so we can put 0 in their respective sales column
closed_ids = test_store["Id"][test["Open"] == 0].values
print(closed_ids.shape)
closed_ids

In [ ]:
# remove all rows(store,date) that were closed
test_store = test_store[test_store["Open"] != 0]

In [ ]:
test_store = test_store.reset_index()

In [ ]:
test_store.drop(["index"],axis =1, inplace= True)

In [ ]:
# define training and testing sets
train = train_store.drop(["Store","Open"],axis=1)

In [ ]:
test = test_store.drop(["Id","Store","Open"],axis=1)

In [ ]:
def load_train_data(scaler_x, scaler_y):
    '''
    Transform train data set and separate a test dataset to validate the model in the end of training and normalize data
    '''
    X_train = train.drop(["Sales"], axis=1) # Features
    y_train = np.array(train["Sales"]).reshape((len(X_train), 1)) # Targets
    X_train = scaler_x.fit_transform(X_train)
    y_train = scaler_y.fit_transform(y_train)

    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

    return (X_train, y_train), (X_test, y_test)

In [ ]:
def load_test_data():
    '''
    Remove column of predictions and normalize data of submission test data set.
    '''
    X_test = test # Features
    X_test = StandardScaler().fit_transform(X_test)

    return X_test

WOW! That took a lot of time, but we finally have clean and perfect data that we want. Now to the model building part.

### Model Building

### Neural Network

In [ ]:
def rmspe_val(y_true, y_pred):
    '''
    RMSPE calculus to validate evaluation metric about the model
    '''
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true), axis=0))[0]
def rmse(y_true, y_pred):
    '''
    RMSE calculus to use during training phase
    '''
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
def rmspe(y_true, y_pred):
    '''
    RMSPE calculus to use during training phase
    '''
    return K.sqrt(K.mean(K.square((y_true - y_pred) / y_true), axis=-1))

In [2]:
# We took Relu activation function as it is from 0 to infinity
def create_model():
    '''
    Create a neural network
    '''
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation="relu", kernel_initializer='normal'))
    model.add(Dropout(0.2)) # We are dropping a few neurons for generalizing the model
    model.add(Dense(32, input_dim=X_train.shape[1], activation="relu", kernel_initializer='normal'))
    model.add(Dropout(0.2))
    model.add(Dense(32, input_dim=X_train.shape[1], activation="relu", kernel_initializer='normal'))
    model.add(Dropout(0.2)) 
    model.add(Dense(1, activation="linear", kernel_initializer='normal'))
    adam = Adam(lr=1e-3, decay=1e-3)

    # Compile model
    model.compile(loss="mean_squared_error", optimizer=adam, metrics=[rmse, rmspe])

    return model

In [ ]:
# Hyperparameters and load data to train the model
batch_size = 32
nb_epoch = 100

scaler_x = StandardScaler()
scaler_y = StandardScaler()

print('Loading data...')
(X_train, y_train), (X_test, y_test) = load_train_data(scaler_x, scaler_y)

print('Build model...')
model = create_model()
model.summary()

In [ ]:
print('Fit model...')
filepath="weights_rossmann.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
callbacks_list = [checkpoint, early_stopping]

log = model.fit(X_train, y_train,
          validation_split=0.20, batch_size=batch_size, epochs=nb_epoch, shuffle=True, callbacks=callbacks_list)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def show_info(model, X, y, log, weights = None):
    '''
    Show metrics about the evaluation model and plots about loss, rmse and rmspe
    '''
    if (log != None):
        # summarize history for loss
        plt.figure(figsize=(14,10))
        plt.plot(log.history['loss'])
        plt.plot(log.history['val_loss'])
        plt.title('Model Loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()
        print('\n')
        
        # summarize history for rmse
        plt.figure(figsize=(14,10))
        plt.plot(log.history['rmse'])
        plt.plot(log.history['val_rmse'])
        plt.title('Model RMSE')
        plt.ylabel('rmse')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()
        print('\n')
        
        # summarize history for rmspe
        plt.figure(figsize=(14,10))
        plt.plot(log.history['rmspe'])
        plt.plot(log.history['val_rmspe'])
        plt.title('Model RMSPE')
        plt.ylabel('rmspe')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

    if (weights != None):
        model.load_weights(weights)

    predictions = model.predict(X, verbose=1)

    mse = mean_squared_error(y, predictions)
    rmse = sqrt(mse)
    rmspe = rmspe_val(y, predictions)

    print('MSE: %.3f' % mse)
    print('RMSE: %.3f' % rmse)
    print('RMSPE: %.3f' % rmspe)

In [ ]:
show_info(model, X_test, y_test, log, weights='weights_rossmann.best.hdf5')

In [ ]:
test_data = load_test_data()
predict = model.predict(test_data)
predict = scaler_y.inverse_transform(predict)

In [ ]:
pred=pd.DataFrame(predict,columns = ['Sales'])
submission = pd.concat([test_store['Id'],pred],axis=1)
# Creating closed stores dataframe
Closed_Stores = pd.DataFrame(closed_ids,columns = ['Id'])
print(Closed_Stores.shape)
Closed_Stores['Sales'] = 0
submission = submission.append(Closed_Stores)
# Converting it to csv
submission.to_csv('submission.csv', index=False)

#### We submitted and got an error of 0.18